In [1]:
import  pandas as pd
import numpy as np
df = pd.read_csv('merged_df_abstract_preprocess_all.csv')
df = df.dropna(subset=['processed_abstract'])

C:\Users\cshch\AppData\Local\Temp\ipykernel_18780\2488393254.py:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('merged_df_abstract_preprocess_all.csv')


In [2]:
print(df.shape)
print(df.columns)

(311557, 20)
Index(['Title', 'Abstract', 'Disciplines', 'Keywords', 'Source',
       'Disciplines_split', 'Astronomy', 'Biology', 'Chemistry',
       'Computer Science', 'Economics', 'Education & instruction',
       'Electrical Engineering', 'European & international law', 'History',
       'Mathematics', 'Physics', 'Social Sciences', 'Statistics',
       'processed_abstract'],
      dtype='object')


In [3]:
# Preprocessing
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

# Vectorize the Text Data using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['processed_abstract'])
y = df['Disciplines']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# 3 RandomForest
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import joblib

model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train, y_train)
joblib.dump(model_rf, 'model/3_Model_RF.joblib')

y_pred = model_rf.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.7920304275259982


C:\Users\cshch\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\cshch\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification Report:
                                                      precision    recall  f1-score   support

                                          Astronomy       0.93      0.40      0.56      1887
                                  Astronomy/Physics       0.80      0.73      0.76      2640
                                            Biology       0.97      0.23      0.38       292
                                  Chemistry/Physics       0.00      0.00      0.00       291
                                   Computer Science       0.74      0.98      0.85     24139
            Computer Science/Electrical Engineering       0.84      0.01      0.02      2745
Computer Science/Electrical Engineering/Mathematics       0.52      0.03      0.06       539
                       Computer Science/Mathematics       0.81      0.02      0.05       687
                        Computer Science/Statistics       0.00      0.00      0.00       404
                                          Eco

C:\Users\cshch\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [7]:
import pandas as pd

n_top_keywords = 100
feature_names = vectorizer.get_feature_names_out()

keywords_dict = {}

# Using feature importances for RandomForest instead of coef_
for i, label in enumerate(model_rf.classes_):
    # Extract the feature importances for each class
    importances = model_rf.feature_importances_
    top_keywords_idx = importances.argsort()[-n_top_keywords:][::-1]

    top_keywords = [(feature_names[idx], importances[idx]) for idx in top_keywords_idx]
    keywords_dict[label] = top_keywords

keywords_df = pd.DataFrame.from_dict(keywords_dict, orient='index')
keywords_df.to_csv("keywords_from_model/3_keywords_RandomForest.csv")

# Couldn't provide the keywords for each specific category